# Multi-Channel Contacts

In [1]:
!unzip "/content/Dataset for Multi-Channel Contacts Problem (3).zip"

Archive:  /content/Dataset for Multi-Channel Contacts Problem (3).zip
  inflating: contacts.json           


In [2]:
import json
from tqdm.notebook import tqdm

In [3]:
with open('contacts.json') as f:
    data = json.load(f)

In [4]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data)
df = df.replace('', np.NaN)

In [5]:
df

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,NaN,1,NaN
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,NaN,9125983679,0,NaN
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,NaN,300364407,2,NaN
...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku


In [6]:
email_group = df.groupby('Email').Id.agg(lambda x: set(x))
phone_group = df.groupby('Phone').Id.agg(lambda x: set(x))
order_group = df.groupby('OrderId').Id.agg(lambda x: set(x))

In [7]:
d = {i: set() for i in df.Id}

In [8]:
for ids in email_group:
    for id in ids:
        d[id] |= set(ids)
for ids in phone_group:
    for id in ids:
        d[id] |= set(ids)
for ids in order_group:
    for id in ids:
        d[id] |= set(ids)


In [9]:
for i in tqdm(range(3)):
    for id, ids in d.items():
        for id_ in list(ids):
            d[id] |= d[id_]

In [10]:
id_to_contact = df.set_index('Id').Contacts.to_dict()

def get_sum_contact(ids_set):
    return sum([id_to_contact[id] for id in ids_set])

In [11]:
df['set'] = df.Id.apply(lambda x: d[x])
df['trace'] = df.set.apply(lambda x: '-'.join(map(str, sorted(list(x)))))
df['n_con'] = df.set.apply(lambda x: str(get_sum_contact(x)))
df

,Id,Email,Phone,Contacts,OrderId,set,trace,n_con
0,0,gkzAbIy@qq.com,NaN,1,NaN,{0},0,1
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,"{1, 165605, 2458, 140081, 115061, 98519, 476346}",1-2458-98519-115061-140081-165605-476346,12
2,2,NaN,9125983679,0,NaN,"{159312, 2, 348955, 322639}",2-159312-322639-348955,4
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,{3},3,0
4,4,NaN,300364407,2,NaN,{4},4,2
...,...,...,...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv,{499995},499995,2
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS,{499996},499996,4
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO,{499997},499997,2
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku,"{499998, 121111}",121111-499998,5


In [12]:
df['out'] = df.trace + ', ' + df.n_con

In [13]:
out = df[['Id', 'out']]
out.columns = ['ticket_id', 'ticket_trace/contact']
out.to_csv('out.csv', index=False)

In [14]:
out

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
499995,499995,"499995, 2"
499996,499996,"499996, 4"
499997,499997,"499997, 2"
499998,499998,"121111-499998, 5"


# Recheck

In [16]:
out.nunique()

ticket_id               500000
ticket_trace/contact    291919
dtype: int64

In [17]:
df

,Id,Email,Phone,Contacts,OrderId,set,trace,n_con,out
0,0,gkzAbIy@qq.com,NaN,1,NaN,{0},0,1,"0, 1"
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,"{1, 165605, 2458, 140081, 115061, 98519, 476346}",1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,NaN,9125983679,0,NaN,"{159312, 2, 348955, 322639}",2-159312-322639-348955,4,"2-159312-322639-348955, 4"
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,{3},3,0,"3, 0"
4,4,NaN,300364407,2,NaN,{4},4,2,"4, 2"
...,...,...,...,...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv,{499995},499995,2,"499995, 2"
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS,{499996},499996,4,"499996, 4"
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO,{499997},499997,2,"499997, 2"
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku,"{499998, 121111}",121111-499998,5,"121111-499998, 5"


In [18]:
df[df.trace == '1-2458-98519-115061-140081-165605-476346']

,Id,Email,Phone,Contacts,OrderId,set,trace,n_con,out
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,"{1, 165605, 2458, 140081, 115061, 98519, 476346}",1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"
2458,2458,ULziZaVD@hotmail.com,069988936,1,vDDJJcxfLtSfkooPhbYnJdxov,"{1, 165605, 2458, 140081, 115061, 98519, 476346}",1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"
98519,98519,ULziZaVD@hotmail.com,NaN,2,mwVhJZGKtahXEdLMwVLcOAxXG,"{1, 165605, 476346, 140081, 115061, 98519, 2458}",1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"
115061,115061,JmMSyjzmxdelSmeAHBUi@yahoo.com,069988936,4,NaN,"{1, 165605, 476346, 140081, 115061, 98519, 2458}",1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"
140081,140081,xXwrpkygOe@yahoo.com,NaN,1,NaN,"{1, 165605, 476346, 140081, 115061, 98519, 2458}",1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"
165605,165605,xXwrpkygOe@yahoo.com,NaN,0,mwVhJZGKtahXEdLMwVLcOAxXG,"{1, 165605, 2458, 140081, 115061, 98519, 476346}",1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"
476346,476346,WXJDcOYGapCzchhwH@gmail.com,NaN,0,vDDJJcxfLtSfkooPhbYnJdxov,"{1, 165605, 2458, 140081, 115061, 98519, 476346}",1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"


In [19]:
df[df.trace.str.len() == df.trace.str.len().max()]

,Id,Email,Phone,Contacts,OrderId,set,trace,n_con,out
2532,2532,NaN,61973086131,4,NaN,"{93186, 94469, 452615, 468746, 101132, 293133,...",2532-5675-19137-33191-63223-93186-94469-100008...,99,2532-5675-19137-33191-63223-93186-94469-100008...
63223,63223,NaN,310004021436,4,tHFMwBQtwfZgUlhBNfQzyKTrv,"{93186, 94469, 452615, 468746, 101132, 293133,...",2532-5675-19137-33191-63223-93186-94469-100008...,99,2532-5675-19137-33191-63223-93186-94469-100008...
93186,93186,yjOUQdf@qq.com,88957892419,4,NaN,"{93186, 94469, 452615, 468746, 101132, 293133,...",2532-5675-19137-33191-63223-93186-94469-100008...,99,2532-5675-19137-33191-63223-93186-94469-100008...
114378,114378,rFnvPCRj@hotmail.com,NaN,1,UnvUtPhkbMwFGcTVkbQdslpOB,"{93186, 94469, 452615, 468746, 101132, 293133,...",2532-5675-19137-33191-63223-93186-94469-100008...,99,2532-5675-19137-33191-63223-93186-94469-100008...
120876,120876,btSoCnAUfaEWCoN@yahoo.com,NaN,4,uAJonQIbDYzoHKCxvfHXGPFoX,"{93186, 94469, 452615, 468746, 101132, 293133,...",2532-5675-19137-33191-63223-93186-94469-100008...,99,2532-5675-19137-33191-63223-93186-94469-100008...
145724,145724,NaN,7193282440,2,klvIALruYswtdsMRMPVCKnqYH,"{93186, 94469, 452615, 468746, 101132, 293133,...",2532-5675-19137-33191-63223-93186-94469-100008...,99,2532-5675-19137-33191-63223-93186-94469-100008...
155038,155038,mtDVsOyjBcJ@gmail.com,NaN,0,LOpRxVbNNpKzhxmLVbkbTVvfv,"{93186, 94469, 452615, 468746, 101132, 302093,...",2532-5675-19137-33191-63223-93186-94469-100008...,99,2532-5675-19137-33191-63223-93186-94469-100008...
157021,157021,lOJWkVIZb@gmail.com,949800773,1,sZZSYzCnyeKOJOALugvWGOBVG,"{93186, 94469, 452615, 468746, 101132, 293133,...",2532-5675-19137-33191-63223-93186-94469-100008...,99,2532-5675-19137-33191-63223-93186-94469-100008...
159958,159958,XteTGTEvwO@hotmail.com,90626552273,1,KefXpOsZDvbrnfODTXopNnaNH,"{93186, 94469, 452615, 468746, 101132, 293133,...",2532-5675-19137-33191-63223-93186-94469-100008...,99,2532-5675-19137-33191-63223-93186-94469-100008...
160596,160596,NaN,031594111,1,lXMgkoPjkBBezndERjTNyJlLO,"{93186, 94469, 452615, 468746, 101132, 302093,...",2532-5675-19137-33191-63223-93186-94469-100008...,99,2532-5675-19137-33191-63223-93186-94469-100008...
